---------------------------------------------------------------------------------------------------------------

#  -------- ES335- MACHINE LEARNING  ASSIGNMENT - 1 --------

-----------------------------------------------------------------------------------------------------

#                                        ------- TASK-3 ------- 

---------------------------------------------------------------------------------------------------------

# Q.1 Demonstrate how to use Zero-Shot Learning and Few-Shot Learning to classify human activities based on the featurized accelerometer data. Qualitatively demonstrate the performance of Few-Shot Learning with Zero-Shot Learning. Which method performs better? Why?

---------------------------------------------------------------------------------------------------------

In [5]:
import os
import pandas as pd
from sklearn.utils import shuffle

def load_data(folder_path):
    data = []
    labels = []
    for class_folder in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_folder)
        for file in os.listdir(class_path):
            file_path = os.path.join(class_path, file)
            df = pd.read_csv(file_path)
            data.append(df.values)
            labels.append(class_folder)
    return data, labels

train_data, train_labels = load_data(r'D:\Users\abhay\Downloads\ES335-Assignment-1-1\Combined\Train')
test_data, test_labels = load_data(r'D:\Users\abhay\Downloads\ES335-Assignment-1-1\Combined\Test')

train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
test_data, test_labels = shuffle(test_data, test_labels, random_state=42)



In [6]:
from langchain_groq.chat_models import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

groq_api_key = os.getenv("API_FINAL_KEY")

# Initialize the chat model with specific settings
chat = ChatGroq(temperature=0.1, groq_api_key=groq_api_key, model_name="llama-3.1-70b-versatile")

# Define the system prompt for Zero-Shot Learning
system = "Demonstrate how to use Zero-Shot Learning to classify human activities based on the featurized accelerometer data. Give just predicted activity only, nothing else."

# Define the human prompt template
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

# Create the chain that will process each input
chain = prompt | chat

# Loop through the test data to make predictions and print the results
for i in range(5):  
    data_summary = f"Sample data: {test_data[i].flatten().tolist()}..."  
    result = chain.invoke({"text": data_summary})
    
    # Output the predicted activity based on Zero-Shot Learning
    print(f"Predicted Activity for sample {i+1} using Zero-Shot Learning: {result}\n")
    print(f"Actual Activity for sample {i+1}: {test_labels[i]}\n")


Predicted Activity for sample 1 using Zero-Shot Learning: content='Walking' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 78009, 'total_tokens': 78011, 'completion_time': 0.0080612, 'prompt_time': 18.485065554, 'queue_time': 20.129402244, 'total_time': 18.493126754}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'stop', 'logprobs': None} id='run-9669fe6d-2604-461b-a350-f6e5f254189c-0' usage_metadata={'input_tokens': 78009, 'output_tokens': 2, 'total_tokens': 78011}

Actual Activity for sample 1: STANDING

Predicted Activity for sample 2 using Zero-Shot Learning: content='Walking' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 62965, 'total_tokens': 62967, 'completion_time': 0.008304642, 'prompt_time': 15.253448142, 'queue_time': 0.015097390999999405, 'total_time': 15.261752784}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'st

In [7]:
from langchain_groq.chat_models import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

groq_api_key = os.getenv("API_FINAL_KEY")


# Initialize the chat model with specific settings
chat = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama-3.1-70b-versatile")

# Define the system prompt for Few-Shot Learning
system = "Demonstrate how to use Few-Shot Learning to classify human activities based on the featurized accelerometer data. Give just predicted activity only, nothing else."

# Define the human prompt template
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

# Create the chain that will process each input
chain = prompt | chat

# Prepare examples for Few-Shot Learning
examples = ""
for i in range(3):  # Use first 3 samples as examples
    data_summary = f"Data: {train_data[i].flatten().tolist()[:20]} => Activity: {train_labels[i]}"
    examples += data_summary + "\n"

# Loop through the test data to make predictions and print the results
for i in range(5):  
    new_data_summary = f"New data: {test_data[i].flatten().tolist()[:20]}"
    few_shot_prompt = f"Here are some examples:\n{examples}\nNow classify the following:\n{new_data_summary}"
    
    # Get the result from Few-Shot Learning model
    result = chain.invoke({"text": few_shot_prompt})
    
    # Output the predicted activity based on Few-Shot Learning
    print(f"Predicted Activity for sample {i+1} using Few-Shot Learning: {result}\n")
    print(f"Actual Activity for sample {i+1}: {test_labels[i]}\n")


Predicted Activity for sample 1 using Few-Shot Learning: content='WALKING' response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 651, 'total_tokens': 655, 'completion_time': 0.016285594, 'prompt_time': 0.194076009, 'queue_time': 6.882396721, 'total_time': 0.210361603}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'stop', 'logprobs': None} id='run-3e83acca-683f-4593-803f-238ab9ec2156-0' usage_metadata={'input_tokens': 651, 'output_tokens': 4, 'total_tokens': 655}

Actual Activity for sample 1: STANDING

Predicted Activity for sample 2 using Few-Shot Learning: content='WALKING_UPSTAIRS' response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 654, 'total_tokens': 662, 'completion_time': 0.032, 'prompt_time': 0.186586116, 'queue_time': 0.005273799999999995, 'total_time': 0.218586116}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'stop', 'logprobs

# Comparing Zero Shot and Few Shot


In [8]:
from langchain_groq.chat_models import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from sklearn.metrics import accuracy_score

groq_api_key = os.getenv("API_FINAL_KEY")


# Initialize the chat model with specific settings
chat = ChatGroq(temperature=0.1, groq_api_key= groq_api_key, model_name="llama-3.1-70b-versatile")

# Define the system prompts
zero_shot_system = "Demonstrate how to use Zero-Shot Learning to classify human activities based on the featurized accelerometer data. Give just predicted activity only, nothing else."
few_shot_system = "Demonstrate how to use Few-Shot Learning to classify human activities based on the featurized accelerometer data. Give just predicted activity only, nothing else."

# Define the human prompt template
human = "{text}"

# Create the chains for Zero-Shot and Few-Shot Learning
zero_shot_prompt = ChatPromptTemplate.from_messages([("system", zero_shot_system), ("human", human)])
few_shot_prompt = ChatPromptTemplate.from_messages([("system", few_shot_system), ("human", human)])

zero_shot_chain = zero_shot_prompt | chat
few_shot_chain = few_shot_prompt | chat

# Prepare examples for Few-Shot Learning
examples = ""
for i in range(3):  # Use first 3 samples as examples
    data_summary = f"Data: {train_data[i].flatten().tolist()[:20]} => Activity: {train_labels[i]}"
    examples += data_summary + "\n"

# Initialize lists to store predictions
zero_shot_predictions = []
few_shot_predictions = []

# Loop through the test data to make predictions and compare results
for i in range(5):  
    # Zero-Shot Learning
    zero_shot_data_summary = f"Sample data: {test_data[i].flatten().tolist()}..."  
    zero_shot_result = zero_shot_chain.invoke({"text": zero_shot_data_summary}).content.strip()
    zero_shot_predictions.append(zero_shot_result)

    # Few-Shot Learning
    new_data_summary = f"New data: {test_data[i].flatten().tolist()[:20]}"
    few_shot_prompt_text = f"Here are some examples:\n{examples}\nNow classify the following:\n{new_data_summary}"
    few_shot_result = few_shot_chain.invoke({"text": few_shot_prompt_text}).content.strip()
    few_shot_predictions.append(few_shot_result)

    # Output the comparison for the current sample
    print(f"Sample {i+1} Comparison:")
    print(f"Zero-Shot Predicted Activity: {zero_shot_result}")
    print(f"Few-Shot Predicted Activity: {few_shot_result}")
    print(f"Actual Activity: {test_labels[i]}\n")

# Calculate accuracy for Zero-Shot Learning
zero_shot_accuracy = accuracy_score(test_labels[:5], zero_shot_predictions)
print(f"Zero-Shot Learning Accuracy: {zero_shot_accuracy * 100:.2f}%")

# Calculate accuracy for Few-Shot Learning
few_shot_accuracy = accuracy_score(test_labels[:5], few_shot_predictions)
print(f"Few-Shot Learning Accuracy: {few_shot_accuracy * 100:.2f}%")


Sample 1 Comparison:
Zero-Shot Predicted Activity: Walking
Few-Shot Predicted Activity: WALKING
Actual Activity: STANDING

Sample 2 Comparison:
Zero-Shot Predicted Activity: Walking
Few-Shot Predicted Activity: WALKING_UPSTAIRS
Actual Activity: WALKING_UPSTAIRS

Sample 3 Comparison:
Zero-Shot Predicted Activity: Walking
Few-Shot Predicted Activity: WALKING_UPSTAIRS
Actual Activity: WALKING_UPSTAIRS

Sample 4 Comparison:
Zero-Shot Predicted Activity: Walking
Few-Shot Predicted Activity: WALKING
Actual Activity: SITTING

Sample 5 Comparison:
Zero-Shot Predicted Activity: Walking
Few-Shot Predicted Activity: WALKING
Actual Activity: WALKING_DOWNSTAIRS

Zero-Shot Learning Accuracy: 0.00%
Few-Shot Learning Accuracy: 40.00%


---------------------------------------------------------------------------------------------------------

# Q.2 Quantitatively compare the accuracy of Few-Shot Learning with Decision Trees (You may use a subset of the test set if you encounter rate-limiting issues). Which method performs better?

---------------------------------------------------------------------------------------------------------

In [9]:
# Select a subset of the test data
subset_size = 50  # Adjust if necessary
test_data_subset = test_data[:subset_size]
test_labels_subset = test_labels[:subset_size]


In [10]:
# Prepare a few-shot learning example using the first 3 training samples
examples = ""
for i in range(3):
    data_summary = f"Data: {train_data[i].flatten().tolist()[:20]} => Activity: {train_labels[i]}"
    examples += data_summary + "\n"


In [12]:
import os
from langchain_groq.chat_models import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from sklearn.metrics import accuracy_score

groq_api_key = os.getenv("API_FINAL_KEY")


# Initialize the chat model with specific settings
chat = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama-3.1-70b-versatile")

# Define the system prompt for Few-Shot Learning
system = "Demonstrate how to use Few-Shot Learning to classify human activities based on the featurized accelerometer data. Give just predicted activity only, nothing else."

# Define the human prompt template
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

# Create the chain that will process each input
chain = prompt | chat

# Prepare examples for Few-Shot Learning
examples = ""
for i in range(3):  # Use first 3 samples as examples
    data_summary = f"Data: {train_data[i].flatten().tolist()[:20]} => Activity: {train_labels[i]}"
    examples += data_summary + "\n"

# Loop through the test data to make predictions and print the results
for i in range(5):  
    new_data_summary = f"New data: {test_data[i].flatten().tolist()[:20]}"
    few_shot_prompt = f"Here are some examples:\n{examples}\nNow classify the following:\n{new_data_summary}"

    # Get the result from Few-Shot Learning model
    result = chain.invoke({"text": few_shot_prompt})
    
    # Output the predicted activity based on Few-Shot Learning
    print(f"Predicted Activity for sample {i+1} using Few-Shot Learning: {result}\n")
    print(f"Actual Activity for sample {i+1}: {test_labels[i]}\n")


# Calculate accuracy for Few-Shot Learning
few_shot_accuracy = accuracy_score(test_labels[:5], few_shot_predictions)
print(f"Few-Shot Learning Accuracy: {few_shot_accuracy * 100:.2f}%")


Predicted Activity for sample 1 using Few-Shot Learning: content='WALKING' response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 651, 'total_tokens': 655, 'completion_time': 0.01600399, 'prompt_time': 0.162329604, 'queue_time': 0.143937122, 'total_time': 0.178333594}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'stop', 'logprobs': None} id='run-e1b19835-b2ef-48af-a506-c21b58d2bc84-0' usage_metadata={'input_tokens': 651, 'output_tokens': 4, 'total_tokens': 655}

Actual Activity for sample 1: STANDING

Predicted Activity for sample 2 using Few-Shot Learning: content='WALKING_UPSTAIRS' response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 654, 'total_tokens': 662, 'completion_time': 0.032, 'prompt_time': 0.1744633, 'queue_time': 3.125263157, 'total_time': 0.2064633}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'stop', 'logprobs': None} id='r

ValueError: Found input variables with inconsistent numbers of samples: [5, 0]

DECISION TREE

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

def flatten_data(data, max_length=None):
    flattened_data = []
    if max_length is None:
        max_length = max(len(np.array(sample).flatten()) for sample in data)
    for sample in data:
        flattened_sample = np.array(sample).flatten()
        if len(flattened_sample) < max_length:
            # Pad the flattened sample to the max length with zeros
            flattened_sample = np.pad(flattened_sample, (0, max_length - len(flattened_sample)), mode='constant')
        flattened_data.append(flattened_sample)
    return np.array(flattened_data), max_length

# Flatten and ensure consistent shapes with the same max_length
X_train_flat, max_length = flatten_data(train_data)
X_test_flat, _ = flatten_data(test_data_subset, max_length=max_length)

# Print shapes to debug
print(f"Shape of X_train_flat: {X_train_flat.shape}")
print(f"Shape of X_test_flat: {X_test_flat.shape}")

# Verify that the number of features match
assert X_train_flat.shape[1] == X_test_flat.shape[1], \
    f"Feature mismatch: X_train has {X_train_flat.shape[1]} features, but X_test has {X_test_flat.shape[1]} features."

# Convert labels to numpy arrays
y_train = np.array(train_labels)
y_test = np.array(test_labels_subset)

# Split training data for validation
X_train, X_val, y_train, y_val = train_test_split(X_train_flat, y_train, test_size=0.2, random_state=42)

# Initialize and train the Decision Tree model
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Predict on the subset of the test data
y_pred = clf.predict(X_test_flat)

# Calculate Decision Tree accuracy
decision_tree_accuracy = accuracy_score(y_test, y_pred) * 100
print(f"Decision Tree Accuracy: {decision_tree_accuracy:.2f}%")



Shape of X_train_flat: (126, 18240)
Shape of X_test_flat: (50, 18240)
Decision Tree Accuracy: 56.00%


In [14]:
if few_shot_accuracy > decision_tree_accuracy:
    print("Few-Shot Learning performs better than Decision Trees.")
elif few_shot_accuracy < decision_tree_accuracy:
    print("Decision Trees perform better than Few-Shot Learning.")
else:
    print("Few-Shot Learning and Decision Trees have the same accuracy.")


Decision Trees perform better than Few-Shot Learning.


---------------------------------------------------------------------------------------------------------

# Q.3  What are the limitations of Zero-Shot Learning and Few-Shot Learning in the context of classifying human activities based on featurized accelerometer data?

---------------------------------------------------------------------------------------------------------

In [15]:
from langchain_groq.chat_models import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

groq_api_key = os.getenv('API_FINAL_KEY')

chat = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama-3.1-70b-versatile")

system = "You are a helpful assistant. Provide detailed explanations in a paragraph"
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke({"text": "What are the limitations of Zero-Shot Learning and Few-Shot Learning in the context of classifying human activities based on featurized accelerometer data?"})

AIMessage(content='Zero-Shot Learning (ZSL) and Few-Shot Learning (FSL) are both machine learning approaches that aim to classify data with limited or no labeled examples. However, in the context of classifying human activities based on featurized accelerometer data, both ZSL and FSL have several limitations. One major limitation of ZSL is that it relies heavily on the semantic relationships between classes, which can be challenging to define and capture in the case of human activities. For instance, the semantic difference between "walking" and "jogging" may not be well-represented in the feature space, leading to poor performance. Additionally, ZSL assumes that the feature space is well-represented by the seen classes, which may not be the case when dealing with diverse human activities. FSL, on the other hand, requires a small number of labeled examples for each new class, which can still be challenging to obtain, especially in cases where data collection is expensive or time-consum

Ans. Zero-Shot Learning (ZSL) and Few-Shot Learning (FSL) are both machine learning approaches that aim to classify data with limited or no labeled examples. However, in the context of classifying human activities based on featurized accelerometer data, both ZSL and FSL have several limitations. One major limitation of ZSL is that it relies heavily on the semantic relationships between classes, which can be challenging to define and may not always be accurate. For example, in activity recognition, the semantic relationship between "walking" and "running" may not be well-defined, leading to poor performance. Additionally, ZSL assumes that the test data is drawn from the same distribution as the training data, which may not be the case in real-world scenarios where new activities or variations of existing activities may be encountered. Few-Shot Learning, on the other hand, requires a small number of labeled examples for each new class, which can still be challenging to obtain, especially in cases where data collection is time-consuming or expensive. Furthermore, FSL models can suffer from overfitting to the few available examples, leading to poor generalization performance. Another limitation of both ZSL and FSL is that they often rely on pre-trained models or feature extractors, which may not be optimized for the specific activity recognition task at hand. Moreover, the featurized accelerometer data may not capture the nuances of human activities, leading to poor performance. Finally, both ZSL and FSL may not be able to handle the variability and complexity of human activities, such as the differences in movement patterns, speeds, and intensities, which can lead to poor classification performance.

---------------------------------------------------------------------------------------------------------

# Q.4 What does the model classify when given input from an entirely new activity that it hasn't seen before? 

---------------------------------------------------------------------------------------------------------

In [ ]:
from langchain_groq.chat_models import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

groq_api_key = os.getenv('API_FINAL_KEY')

chat = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama-3.1-70b-versatile")

system = "You are a helpful assistant. Provide detailed explanations in a paragraph"
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke({"text": "What does the llm model classify when given input from an entirely new activity that it hasn't seen before? "})

AIMessage(content="When given input from an entirely new activity that it hasn't seen before, a Large Language Model (LLM) like myself will attempt to classify it based on patterns and relationships learned from the vast amount of text data I was trained on. Since I haven't seen this specific activity before, I won't have any direct knowledge or explicit classification rules to apply. However, I will use various techniques to make an educated guess. I'll analyze the input text for contextual clues, such as keywords, phrases, and sentence structures, to identify potential connections to known concepts or categories. I might also rely on my understanding of linguistic patterns, such as verb tenses, noun phrases, and semantic roles, to infer the activity's characteristics. Additionally, I may use analogies or metaphors to relate the new activity to something I've seen before, even if it's not an exact match. Ultimately, my classification will be a probabilistic prediction, and I may provi

Ans. When given input from an entirely new activity that it hasn\'t seen before, the LLM (Large Language Model) model will attempt to classify it based on patterns and relationships learned from its vast training data. Since the model has not been explicitly trained on this new activity, it will rely on its ability to generalize and make inferences. The model will analyze the input text and try to identify similarities with known activities or concepts it has learned about during training. This process is called "zero-shot learning" or "out-of-distribution generalization." The model may use various techniques, such as semantic role labeling, entity recognition, or part-of-speech tagging, to understand the context and meaning of the input text. If the model is unable to find a clear match with a known activity, it may classify the input as "unknown" or "unseen," or it may generate a response that indicates it is uncertain or lacks sufficient information to provide a confident classification. However, some advanced LLMs may also use techniques like "few-shot learning" or "meta-learning" to adapt quickly to new activities and learn from a few examples, even if they have not seen them before.

---------------------------------------------------------------------------------------------------------

# Q.5 Test the model with random data (ensuring the data has the same dimensions and range as the previous input) and report the results. 

---------------------------------------------------------------------------------------------------------

In [17]:
from langchain_groq.chat_models import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from sklearn.metrics import accuracy_score

groq_api_key = os.getenv("API_FINAL_KEY")


# Initialize the chat model with specific settings
chat = ChatGroq(temperature=0.1, groq_api_key= groq_api_key, model_name="llama-3.1-70b-versatile")

# Define the system prompts
zero_shot_system = "Zero-Shot Learning to classify human activities based on the featurized accelerometer data."
few_shot_system = "Few-Shot Learning to classify human activities based on the featurized accelerometer data."

# Define the human prompt template
human = "{text}"

# Create the chains for Zero-Shot and Few-Shot Learning
zero_shot_prompt = ChatPromptTemplate.from_messages([("system", zero_shot_system), ("human", human)])
few_shot_prompt = ChatPromptTemplate.from_messages([("system", few_shot_system), ("human", human)])

zero_shot_chain = zero_shot_prompt | chat
few_shot_chain = few_shot_prompt | chat


# Example of synthetic train and test data (replace with actual data)
train_data = np.random.rand(100, 126)  # 100 samples, 126 features each
train_labels = np.random.choice(["walking", "standing", "sitting"], 100)  # Random labels

test_data = np.random.rand(10, 126)  # 10 samples, 126 features each
test_labels = np.random.choice(["walking", "standing", "sitting"], 10)  # Corresponding labels

# Prepare examples for Few-Shot Learning
examples = ""
for i in range(3):  # Use first 3 samples as examples
    data_summary = f"Data: {train_data[i].tolist()} => Activity: {train_labels[i]}"
    examples += data_summary + "\n"

# Generate random test data with the same dimensions and range
random_test_data = np.random.uniform(low=np.min(train_data), high=np.max(train_data), size=test_data.shape)

# Initialize lists to store predictions
zero_shot_predictions = []
few_shot_predictions = []

# Loop through the test data to make predictions and compare results
zero_shot_predictions = []
few_shot_predictions = []

# Loop through the test data to make predictions and compare results
for i in range(5):  
    # Zero-Shot Learning
    zero_shot_data_summary = f"Sample data: {test_data[i].flatten().tolist()}..."  
    zero_shot_result = zero_shot_chain.invoke({"text": zero_shot_data_summary}).content.strip()
    zero_shot_predictions.append(zero_shot_result)

    # Few-Shot Learning
    new_data_summary = f"New data: {test_data[i].flatten().tolist()[:20]}"
    few_shot_prompt_text = f"Here are some examples:\n{examples}\nNow classify the following:\n{new_data_summary}"
    few_shot_result = few_shot_chain.invoke({"text": few_shot_prompt_text}).content.strip()
    few_shot_predictions.append(few_shot_result)

    # Output the comparison for the current sample
    print(f"Sample {i+1} Comparison:")
    print(f"Zero-Shot Predicted Activity: {zero_shot_result}")
    print(f"Few-Shot Predicted Activity: {few_shot_result}")
    print(f"Actual Activity: {test_labels[i]}\n")

# Calculate accuracy for Zero-Shot Learning
zero_shot_accuracy = accuracy_score(test_labels[:5], zero_shot_predictions)
print(f"Zero-Shot Learning Accuracy: {zero_shot_accuracy * 100:.2f}%")

# Calculate accuracy for Few-Shot Learning
few_shot_accuracy = accuracy_score(test_labels[:5], few_shot_predictions)
print(f"Few-Shot Learning Accuracy: {few_shot_accuracy * 100:.2f}%")


Sample 1 Comparison:
Zero-Shot Predicted Activity: Zero-Shot Learning for Human Activity Classification

Zero-shot learning is a type of machine learning where a model is trained to recognize objects or activities that it has not seen before. In this case, we will use zero-shot learning to classify human activities based on featurized accelerometer data.

Dataset
--------

The dataset consists of featurized accelerometer data, which is a sequence of numbers representing the acceleration of the device in three dimensions (x, y, z). The dataset is labeled with the corresponding human activity (e.g., walking, running, sitting, etc.).

Model
------

We will use a transformer-based model, specifically the BERT (Bidirectional Encoder Representations from Transformers) model, which is pre-trained on a large corpus of text data. We will fine-tune the model on our dataset to adapt it to the task of human activity classification.

Code
-----

```python
import torch
import torch.nn as nn
import t